To dos:
- create a dataset with age and lon, lat and download it - CHECK
- specify a new location to save images on the server -  CHECK
- loop over the dataset - CHECK
 - plug in lat and lon into the api code below - CHECK
 - extract all images for that location - CHECK
 - name image according to age and its unique code after the last - CHECK

CNN
- rewatch simple CNN video how to extract age from name file - CHECK
- run a simple CNN and compare results - CHECK
- how to boost perfromance
- How to increase image quality and coverage?

Extent dataset:
- access Mapillary data either through API or the download you did
- repeat above excercise
- compare R2
- compare coverage of images between the two - are the duplicates?

Integration
- add a building ID to the kartaview key 
- engineer a way to match the age predictions back to the main dataset for those buildings where the age exists
- can you actually predict the age for all buildings, including those images that you have trained it on conceptually speaking?

## Imports

In [28]:
import requests
import re
import json
import os
import pandas as pd

## Data

In [29]:
# Define the path for the processed DataFrame
processed_df_path = "/workspace/workspace/ufo-prediction/demo/kartaview_key.csv"

# Load the existing processed DataFrame if it exists, else create a new one
if os.path.exists(processed_df_path):
    kartaview_keys = pd.read_csv(processed_df_path)
    print("Loaded processed DataFrame from", processed_df_path)
else:
    # Assuming df_subset is initially loaded or defined here
   print("Creating a new processed DataFrame")
   path_data_RCA = os.path.join(dataset.DATA_DIR, 'rca-ufo-merge_ALL.csv')
   df = pd.read_csv(path_data_RCA, encoding='latin1')
   kartaview_keys = df[['lon', 'lat','age_right']]
   kartaview_keys.to_csv('/workspace/workspace/ufo-prediction/demo/kartaview_key.csv', index=False)

print(kartaview_keys)   

Loaded processed DataFrame from /workspace/workspace/ufo-prediction/demo/kartaview_key.csv
          lon        lat  age_right
0   -3.598445  37.170568     1992.0
1   -3.683087  40.423877     1923.0
2   -3.708202  40.417582     1988.0
3   -8.725824  42.240112     1970.0
4   -0.463673  39.493747     2002.0
5   -0.571214  38.339208     2008.0
6   -3.614856  37.200792     2002.0
7   -3.703766  40.416340     1768.0
8   -8.827510  42.562819     1980.0
9    2.193825  41.397634     1880.0
10  -3.697594  40.433163     1935.0
11  -3.703766  40.416340     1768.0
12  -3.698714  40.404475     1920.0
13   2.164025  41.399675     1960.0
14   2.157202  41.384330     1982.0
15   2.154698  41.383069     1930.0
16  -4.660315  40.669031     2013.0
17  -8.724449  42.220811     2006.0
18  -5.272995  36.443914     1988.0
19  -4.791229  37.885040     2003.0
20   2.160491  41.392731     1976.0
21  -0.464190  39.485044     2011.0
22  -3.675819  40.429488     1948.0
23  -4.459373  36.686364     2014.0
24  -4.45

In [30]:
# Set directory

current_directory = '/workspace/workspace/ufo-prediction/image_data'

# Check if the new directory exists, if not, create it
if not os.path.exists(current_directory):
    os.makedirs(current_directory)
    print(f"Directory {current_directory} created.")
else:
    print(f"Directory {current_directory} already exists.")

Directory /workspace/workspace/ufo-prediction/image_data already exists.


In [31]:
for index, row in kartaview_keys.iterrows():
    precision = 6  # Start with 6 decimal places
    success = False  # Flag to indicate if the request was successful

    while precision > 2 and not success:
        # Format lon and lat to the current precision
        lon = f"{row['lon']:.{precision}f}"
        lat = f"{row['lat']:.{precision}f}"

        # Construct the API URL
        url = "https://api.openstreetcam.org/2.0/photo/?lat={}&lng={}".format(lat, lon)

        # Send a GET request to the API
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            url_pattern = r'https://storage\d+\.openstreetcam\.org/files/photo/\d+/\d+/\d+/[^"]+\.jpg'
            urls = re.findall(url_pattern, json.dumps(data))

            filtered_urls = [
                url for url in urls
                if not any(x in url.rsplit('/', 2)[-2] for x in ["{{sizeprefix}}", "proc"]) and
                ("th" in url.rsplit('/', 2)[-2] and not "lth" in url.rsplit('/', 2)[-2])
            ]

            if filtered_urls:
                for image_url in filtered_urls:
                    file_name = "{}_{}".format(row['age_right'], image_url.split('/')[-1])
                    if not file_name.endswith('.jpg'):
                        file_name += '.jpg'
                    file_path = os.path.join(current_directory, file_name)

                    # Check if the file already exists
                    if os.path.exists(file_path):
                        print(f"File already exists: {file_path}. Skipping download.")
                    else:
                        image_response = requests.get(image_url)

                        if image_response.status_code == 200:
                            with open(file_path, 'wb') as f:
                                f.write(image_response.content)
                            print("Image downloaded successfully: {}".format(file_path))
                        else:
                            print("Failed to download the image.")
                success = True  # Mark success as True to exit the while loop
            else:
                print("No suitable images found for location: lon={}, lat={}".format(lon, lat))
                precision -= 1  # Reduce precision by one decimal place
        else:
            print("Failed to retrieve data from the API for location: lon={}, lat={}. Trying with reduced precision.")
            

    # After processing, remove the row from df_subset
    kartaview_keys = kartaview_keys.drop(index)

    # Save the updated DataFrame to a CSV file
    kartaview_keys.to_csv(processed_df_path, index=False)

    if not success:
        print("Unable to retrieve data from the API with sufficient precision for location: lon={}, lat={}".format(row['lon'], row['lat']))


No suitable images found for location: lon=-3.598445, lat=37.170568
No suitable images found for location: lon=-3.59845, lat=37.17057
No suitable images found for location: lon=-3.5984, lat=37.1706
No suitable images found for location: lon=-3.598, lat=37.171
Unable to retrieve data from the API with sufficient precision for location: lon=-3.598445037590166, lat=37.170568138778656
No suitable images found for location: lon=-3.683087, lat=40.423877
No suitable images found for location: lon=-3.68309, lat=40.42388
No suitable images found for location: lon=-3.6831, lat=40.4239
No suitable images found for location: lon=-3.683, lat=40.424
Unable to retrieve data from the API with sufficient precision for location: lon=-3.683086642029149, lat=40.42387659807049
No suitable images found for location: lon=-3.708202, lat=40.417582
No suitable images found for location: lon=-3.70820, lat=40.41758
No suitable images found for location: lon=-3.7082, lat=40.4176
No suitable images found for locati

In [14]:
# Iterate over each row in df_subset
for index, row in kartaview_keys.iterrows():
    # Format lon and lat to ensure they are strings with up to 8 decimal places
    lon = "{:.6f}".format(row['lon']).rstrip('0').rstrip('.')
    lat = "{:.6f}".format(row['lat']).rstrip('0').rstrip('.')

    # Check if lon and lat have less than 8 digits after the decimal
    if len(lon.split('.')[-1]) < 3 or len(lat.split('.')[-1]) < 3:
        print("* Location not specific enough")
        continue

    # Construct the API URL
    url = "https://api.openstreetcam.org/2.0/photo/?lat={}&lng={}".format(lat, lon)
    #url="http://api.openstreetcam.org/2.0/list/nearby-photos/?lat={}&lng={}".format(lat, lon)
    #print(url)

    # Send a GET request to the API
    response = requests.get(url)

    # Proceed if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Define a regular expression pattern to match the URLs
        url_pattern = r'https://storage\d+\.openstreetcam\.org/files/photo/\d+/\d+/\d+/[^"]+\.jpg'

        # Find all matches of the pattern
        urls = re.findall(url_pattern, json.dumps(data))

        # Filter URLs based on the specified conditions
        filtered_urls = [url for url in urls if not any(x in url.rsplit('/', 2)[-2] for x in ["{{sizeprefix}}", "proc"]) and ("th" in url.rsplit('/', 2)[-2] and not "lth" in url.rsplit('/', 2)[-2])]

        # Download images from filtered URLs
        for image_url in filtered_urls:
            file_name = image_url.split('/')[-1]
            if not file_name.endswith('.jpg'):
                file_name += '.jpg'

            # Name the .jpg using age_right followed by the original file_name
            file_name = "{}_{}".format(row['age_right'], file_name)

            # Combine the directory with the filename
            file_path = os.path.join(current_directory, file_name)

            # Send a GET request to fetch the image
            image_response = requests.get(image_url)

            if image_response.status_code == 200:
                # Save the image to a file
                with open(file_path, 'wb') as f:
                    f.write(image_response.content)
                print("Image downloaded successfully: {}".format(file_path))
            else:
                print("Failed to download the image.")
        else:
            print("Failed to retrieve data from the API for location: lon={}, lat={}".format(lon, lat))
        # After processing, remove the row from df_subset
    kartaview_keys = kartaview_keys.drop(index)

    # Save the updated DataFrame to a CSV file
    kartaview_keys.to_csv(processed_df_path, index=False)


Failed to retrieve data from the API for location: lon=4.519975, lat=52.075455
Failed to retrieve data from the API for location: lon=5.970008, lat=50.885978
Failed to retrieve data from the API for location: lon=5.620564, lat=52.349998
Failed to retrieve data from the API for location: lon=4.540407, lat=52.060026
Failed to retrieve data from the API for location: lon=5.619979, lat=52.026556
Failed to retrieve data from the API for location: lon=4.700046, lat=52.295673
Failed to retrieve data from the API for location: lon=5.978112, lat=51.979975
Failed to retrieve data from the API for location: lon=5.07596, lat=52.070016
Failed to retrieve data from the API for location: lon=4.880037, lat=52.3599


KeyboardInterrupt: 

In [12]:

# Define the API endpoint with the specific latitude and longitude
# 13.38953209
base_url = "https://api.openstreetcam.org/2.0/photo/?lat={}&lng={}"


# Send a GET request to the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Debug: Print the response JSON to find the correct key for the image URL
    #print(response.json())

    data = response.json()

    # Define a regular expression pattern to match the URLs
    url_pattern = r'https://storage\d+\.openstreetcam\.org/files/photo/\d+/\d+/\d+/[^"]+\.jpg'

    # Find all matches of the pattern
    urls = re.findall(url_pattern, json.dumps(data))
    #print(urls)

    # Filter URLs that include "{{sizeprefix}}", "proc", or "th" before the last "/" sign
    filtered_urls = []
    for url in urls:
    # Split the URL by "/" to get all segments
        parts = url.split('/')
    # Check if the segment before the last one matches any of the specified patterns
        if len(parts) > 1:  # Ensure there are enough parts to inspect
            segment_to_check = parts[-2]  # Get the segment before the last one
        # Check if the segment does not contain any of the specified patterns
            if not any(pattern in segment_to_check for pattern in ["{{sizeprefix}}", "proc"]) and (segment_to_check.endswith("th") and not segment_to_check.endswith("lth")):
                filtered_urls.append(url)
    
    print(filtered_urls)
    
    if filtered_urls:  
        # Check if image_url is not None
        for image_url in filtered_urls:
            print(image_url)
            file_name = image_url.split('/')[-1]
            if not file_name.endswith('.jpg'):
                file_name += '.jpg'
        # Combine the directory with the filename
            file_path = os.path.join(current_directory, file_name)
        # Send a GET request to fetch the image
            image_response = requests.get(image_url)
    
            if image_response.status_code == 200:
            # Save the image to a file
                with open(file_path, 'wb') as f:
                    f.write(image_response.content)
                print("Image downloaded successfully.")
            else:
                print("Failed to download the image.")
    else:
        print("Image URL was not found in the API response.")
else:
    print("Failed to retrieve data from the API.")



['https://storage2.openstreetcam.org/files/photo/2016/8/25/th/14682_72e79_57bf17a3d0fc7.jpg', 'https://storage6.openstreetcam.org/files/photo/2017/11/10/th/966533_dfcfa_13.jpg', 'https://storage6.openstreetcam.org/files/photo/2017/11/10/th/966541_6cb2b_25.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/7/26/th/2818418_83ecb_5f1d8e994f5a5.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/7/28/th/2827634_9008c_5f20513eb1ff1.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/8/5/th/2865646_84ea2_5f2a2bff88b76.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/8/5/th/2865742_9bb8d_5f2a3576d6d95.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/8/15/th/2917218_41295_5f37ec1e71d2e.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/8/23/th/2943810_b2100_5f41d7658e0a9.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/10/28/th/3196014_31e7b_5f998cea88f8f.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/10/29/th/3198